In [1]:
import numpy as np
import pandas as pd
import os
import glob
import scipy 

In [2]:
path = '/Volumes/LaCie/school/combine_lab/Data/squamous_carcinoma/'

cSCC_counts.txt file is a merged raw count matrix of all patients (after 10x cellranger processing and filtered: cells with <200 genes or >10% mitochondiral gene expression were rm).

Thus, separate by patient and output the saved file:

In [3]:
# cSCC_counts = pd.read_csv(path+'scRNAseq/'+'GSE144236_cSCC_counts.txt', sep = '\t', chunksize=1000)

# # merge all chunks into one df
# scRNAseq = pd.concat([chunk for chunk in cSCC_counts])

# # split df by patient
# cSCC_scRNA = []
# for i, j in scRNAseq.groupby(scRNAseq.columns.str.split("_").str[0], axis=1):
#     cSCC_scRNA.append(j)
    
# # save df by patient idex to separate CSV files
# for i, j in scRNAseq.groupby(scRNAseq.columns.str.split("_").str[0], axis=1):
#     j.to_csv('Data/squamous_carcinoma/scRNAseq_{}.csv'.format(i), header=True, index_label=False)

In [4]:
# squamous carcinoma dataset
path = '/Volumes/LaCie/school/combine_lab/Data/squamous_carcinoma/'

all_counts = pd.read_csv(path+'ST/'+'GSE144239_ST_all_counts.txt', sep = '\t')
st_barcodes = pd.read_csv(path+'ST/'+'GSE144239_ST_barcode_id.txt', sep = '\t', header=None)
visium = pd.read_csv(path+'ST/'+'GSE144239_ST_Visium_counts.txt', sep = '\t')

metadata = pd.read_csv(path+'scRNAseq/'+'GSE144236_patient_metadata_new.txt', sep = '\t')

In [5]:
# read in all patient scRNAseq files
scRNA = [pd.read_csv(j, index_col=0) for j in glob.glob(os.path.join(path, '*.csv'))]

In [6]:
range(len(scRNA)) # scRNA for 10 patients

range(0, 10)

In [7]:
scRNA[0]

,P1_Tumor_AAACCTGAGTCAAGCG,P1_Tumor_AAACCTGCAAATTGCC,P1_Tumor_AAACCTGGTAGGAGTC,P1_Tumor_AAACGGGAGATGTAAC,P1_Tumor_AAACGGGAGCTGCAAG,P1_Tumor_AAAGATGAGAAGGTTT,P1_Tumor_AAAGATGCAATAACGA,P1_Tumor_AAAGATGCAGACTCGC,P1_Tumor_AAAGATGCAGTCAGCC,P1_Tumor_AAAGATGGTGGTGTAG,...,P1_Normal_TTGGCAAGTGTTCTTT,P1_Normal_TTGGCAATCCTGCTTG,P1_Normal_TTGTAGGAGGTGCTTT,P1_Normal_TTGTAGGCAAGAAGAG,P1_Normal_TTGTAGGTCCTGCTTG,P1_Normal_TTTACTGGTCTACCTC,P1_Normal_TTTATGCGTTTGTTGG,P1_Normal_TTTCCTCAGGAGCGAG,P1_Normal_TTTCCTCCACGGCGTT,P1_Normal_TTTCCTCTCACAGTAC
Patient,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
"Tissue: 0=Normal, 1=Tumor",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
AP006222.2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145205.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAGE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CU459201.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC002321.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
all_counts

,P2_7x45_1,P2_8x30_1,P2_8x32_1,P2_8x34_1,P2_8x36_1,P2_8x38_1,P2_8x44_1,P2_8x46_1,P2_9x29_1,P2_9x31_1,...,P5_41x27_3,P5_41x31_3,P5_41x33_3,P5_41x35_3,P5_41x37_3,P5_41x39_3,P5_41x41_3,P5_42x36_3,P5_42x38_3,P5_42x40_3
Patient,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Section replicate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
pixel_x,3282.6,3473.4,3471.9,3474.1,3469.5,3467.4,3471.4,3462.3,3650.9,3649.9,...,9448.4,9446.6,9437.3,9451.4,9451.1,9445.3,9451.1,9634.2,9628.1,9623.4
pixel_y,9536.7,6539.5,6937.9,7338.8,7736.4,8137.2,9337.8,9738.9,6336.8,6737.0,...,6984.0,7809.0,8205.3,8613.4,9011.1,9412.5,9810.7,8815.8,9224.7,9620.7
FO538757.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MAGEA2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MAGEA6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
LCE5A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NPIPA8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ST for P2, 5, 9 and 10 was generated with an older version of technology with larget spot size and center-to-center distance.

In [9]:
st_keep = np.unique(all_counts.loc['Patient'].values)
print(st_keep)
# type(st_keep)
st_keep = st_keep.tolist()
st_keep = [int(i) for i in st_keep] # turn float to int
print(st_keep)

[ 2.  5.  9. 10.]
[2, 5, 9, 10]


In [10]:
np.unique(all_counts.loc['Section replicate'].values)

array([1., 2., 3.])

In [11]:
print(type(scRNA))
print(type(scRNA[0]))

<class 'list'>
<class 'pandas.core.frame.DataFrame'>


Keep only scRNA for patients 2, 5, 9, and 10

In [12]:
scrna = []
for i in range(len(scRNA)):
    if scRNA[i].loc['Patient'].isin(st_keep).any():
        scrna.append(scRNA[i])
len(scrna)

4

In [13]:
scrna[3]

,P9_Tumor_AAACCTGAGTGTACGG,P9_Tumor_AAACCTGCAGCTATTG,P9_Tumor_AAACCTGCATCCGGGT,P9_Tumor_AAACCTGTCGGAAACG,P9_Tumor_AAACGGGAGTGTCCCG,P9_Tumor_AAAGCAAAGATCGATA,P9_Tumor_AAAGTAGCACCATGTA,P9_Tumor_AAAGTAGTCTGGGCCA,P9_Tumor_AAATGCCCAGCTGGCT,P9_Tumor_AAATGCCGTCTTCGTC,...,P9_Normal_TTTGTCAAGCTGATAA,P9_Normal_TTTGTCAAGGTGCTTT,P9_Normal_TTTGTCAAGTTCGATC,P9_Normal_TTTGTCACAAAGCAAT,P9_Normal_TTTGTCACACAACGTT,P9_Normal_TTTGTCAGTTACTGAC,P9_Normal_TTTGTCATCAACACGT,P9_Normal_TTTGTCATCAGTCCCT,P9_Normal_TTTGTCATCGACAGCC,P9_Normal_TTTGTCATCTGTTTGT
Patient,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
"Tissue: 0=Normal, 1=Tumor",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AP006222.2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145205.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAGE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CU459201.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC002321.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
counts = []
for i in range(len(scrna)):
    counts.append(scrna[i].drop(['Patient', 'Tissue: 0=Normal, 1=Tumor'], axis=0)) # delete first two rows

In [15]:
counts[0]

,P10_Tumor_AAACCTGGTCATGCCG,P10_Tumor_AAACCTGTCGACAGCC,P10_Tumor_AAACCTGTCGCTGATA,P10_Tumor_AAACGGGCACTTACGA,P10_Tumor_AAACGGGTCACTTATC,P10_Tumor_AAAGCAAAGACTAGAT,P10_Tumor_AAAGCAAAGAGCTATA,P10_Tumor_AAAGCAAAGATTACCC,P10_Tumor_AAAGCAACAGCTGGCT,P10_Tumor_AAAGCAACATCCCATC,...,P10_Normal_TTTGGTTCACTCTGTC,P10_Normal_TTTGGTTCATCATCCC,P10_Normal_TTTGGTTCATCCCATC,P10_Normal_TTTGGTTTCTCTAAGG,P10_Normal_TTTGTCAAGACAATAC,P10_Normal_TTTGTCAAGAGTCTGG,P10_Normal_TTTGTCAAGTCGTTTG,P10_Normal_TTTGTCACAGTGGGAT,P10_Normal_TTTGTCATCCAGGGCT,P10_Normal_TTTGTCATCGAGGTAG
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AP006222.2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
RP4-669L17.10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RP11-206L10.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145205.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAGE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CU459201.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC002321.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
from scipy.sparse import csr_matrix

def pd2csr(df):
    #from dense to sparse, use df.astype() with a SparseDtype
    spars_dtype = pd.SparseDtype(int, fill_value=0)
    spars_df = df.astype(spars_dtype)

    # convert to coo()
    df_coo = spars_df.sparse.to_coo()

    # convert to col sparse row matrix
    return csr_matrix((df_coo.data, (df_coo.row, df_coo.col)))

In [17]:
spar_count = []
for i in range(len(counts)):
    spar_count.append(pd2csr(counts[i]))

In [18]:
for i, j in enumerate(st_keep):
    scipy.sparse.save_npz(f"Data/squamous_carcinoma/scRNA_counts_P{j}.npz", spar_count[i])

process metadata

In [19]:
keep = []
for i in range(len(counts)):
    keep.append(counts[i].columns.values)

keep = [l.tolist() for l in keep]
keep1list = [i for patient in keep for i in patient]
len(keep1list)

20406

In [20]:
# sanity check
print(len(keep[0]) + len(keep[1]) + len(keep[2]) + len(keep[3]))

20406


In [21]:
# keep only metadata values from patients of interest
meta = metadata.loc[keep1list, ]
meta

,nCount_RNA,nFeature_RNA,patient,tum.norm,level1_celltype,level2_celltype,level3_celltype
P10_Tumor_AAACCTGGTCATGCCG,7921,2630,P10,Tumor,Fibroblast,Fibroblast,Fibroblast
P10_Tumor_AAACCTGTCGACAGCC,10373,2859,P10,Tumor,Fibroblast,Fibroblast,Fibroblast
P10_Tumor_AAACCTGTCGCTGATA,9389,2665,P10,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
P10_Tumor_AAACGGGCACTTACGA,8079,2689,P10,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
P10_Tumor_AAACGGGTCACTTATC,6729,2337,P10,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
...,...,...,...,...,...,...,...
P9_Normal_TTTGTCAGTTACTGAC,9093,2794,P9,Normal,Epithelial,Normal_KC_Diff,Normal_KC_Diff
P9_Normal_TTTGTCATCAACACGT,5882,1593,P9,Normal,Mac,Mac,Mac
P9_Normal_TTTGTCATCAGTCCCT,8054,2101,P9,Normal,Mac,Mac,Mac
P9_Normal_TTTGTCATCGACAGCC,8815,2236,P9,Normal,Epithelial,Normal_KC_Diff,Normal_KC_Diff


In [22]:
meta.to_csv('Data/squamous_carcinoma/scRNAseq/metadata.csv')

10X Visium used for P4 and 6 (not at sc but better resolution than older version).

Note, spatial spot barcodes are also 16-oligo sequence.

In [23]:
visium

,P4_AAACAAGTATCTCCCA-1_1,P4_AAACATTTCCCGGATT-1_1,P4_AAACCTAAGCAGCCGG-1_1,P4_AAACGAGACGGTTGAT-1_1,P4_AAACGGGCGTACGGGT-1_1,P4_AAAGGGATGTAGCAAG-1_1,P4_AAAGTGTGATTTATCT-1_1,P4_AAAGTTGACTCCCGTA-1_1,P4_AAATAGGGTGCTATTG-1_1,P4_AAATCCGATACACGCC-1_1,...,P6_TTGTGTATGCCACCAA-1_2,P6_TTGTGTTTCCCGAAAG-1_2,P6_TTGTTAGCAAATTCGA-1_2,P6_TTGTTCAGTGTGCTAC-1_2,P6_TTGTTGGCAATGACTG-1_2,P6_TTGTTGTGTGTCAAGA-1_2,P6_TTGTTTCACATCCAGG-1_2,P6_TTGTTTCCATACAACT-1_2,P6_TTGTTTGTATTACACG-1_2,P6_TTGTTTGTGTAAATTC-1_2
Patient,4,4,4,4,4,4,4,4,4,4,...,6,6,6,6,6,6,6,6,6,6
Section replicate,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
AL627309.1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL669831.5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LINC00115,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AP001059.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AP001469.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC136616.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC004556.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### No data matches up with the st_barcodes...

In [25]:
# split spot barcode on _ and keep just the ending
spt = []
for i in visium.columns:
    head, sep, tail = i.partition('_')
    spt.append(tail)

# split spot barcode on - and keep just first part
spot = []
for i in spt:
    spot.append(i.split('-', 1)[0])
np.unique(spot)

array(['AAACAAGTATCTCCCA', 'AAACACCAATAACTGC', 'AAACAGAGCGACTCCT', ...,
       'TTGTTTCCATACAACT', 'TTGTTTGTATTACACG', 'TTGTTTGTGTAAATTC'],
      dtype='<U16')

In [26]:
# count number of unique spots
len(np.unique(spot))

3993

In [85]:
vs_spots = np.unique(spot).astype(object)
vs_spots

array(['AAACAAGTATCTCCCA', 'AAACACCAATAACTGC', 'AAACAGAGCGACTCCT', ...,
       'TTGTTTCCATACAACT', 'TTGTTTGTATTACACG', 'TTGTTTGTGTAAATTC'],
      dtype=object)

In [44]:
st_barcodes

,0,1,2
0,GTCCGATATGATTGCCGC,2,6
1,GCGACGCGGCATAGTTTA,2,8
2,ACGTGCGCAAATATGACG,2,10
3,ATAAATGTTCACACTACT,2,12
4,TTGCTCGAATAAATAATT,2,14
...,...,...,...
1929,ATTAGTTATGCTGTTCAC,65,53
1930,GGTGAGACCTGAATTCCA,65,55
1931,CTATGTACTGTTCAAACC,65,57
1932,GGCTAGAACGAATAAACC,65,59


In [86]:
st_barcodes.loc[:, 0].isin(vs_spots)

0       False
1       False
2       False
3       False
4       False
        ...  
1929    False
1930    False
1931    False
1932    False
1933    False
Name: 0, Length: 1934, dtype: bool

In [87]:
st_barcodes[st_barcodes.loc[:, 0].isin(vs_spots)]

,0,1,2


In [33]:
print(np.unique(visium.iloc[0].values)) # find unique patients 
print(np.unique(visium.iloc[1].values)) # find unique replicates 

[4 6]
[1 2]


In [120]:
# sanity check that filtering for barcodes in spots is indeed not the same
fake = pd.DataFrame({'spot': ['GTCCGATATGATTGCCGC', 'GCGACGCGGCATAGTTTA','AAACAAGTATCTCCCA', 'TTGTTTGTATTACACG'], #first 2 entries from st_barcodes, last 2 from vs_spots
                        'X': [0, 0, 0, 0]
                        })
fake.iloc[:, 0].isin(st_barcodes.iloc[:,0])
fake[fake.iloc[:,0].isin(st_barcodes.iloc[:,0])]

,spot,X
0,GTCCGATATGATTGCCGC,0
1,GCGACGCGGCATAGTTTA,0


In [121]:
# same same
st_barcodes.iloc[:,0].isin(fake.iloc[:,0])
st_barcodes[st_barcodes.iloc[:,0].isin(fake.iloc[:,0])]

,0,1,2
0,GTCCGATATGATTGCCGC,2,6
1,GCGACGCGGCATAGTTTA,2,8


In [34]:
metadata

,nCount_RNA,nFeature_RNA,patient,tum.norm,level1_celltype,level2_celltype,level3_celltype
P1_Tumor_AAACCTGAGTCAAGCG,40856,5206,P1,Tumor,Epithelial,Tumor_KC_Diff,Tumor_KC_Diff
P1_Tumor_AAACCTGCAAATTGCC,15511,3468,P1,Tumor,Epithelial,Keratinocyte,Keratinocyte
P1_Tumor_AAACCTGGTAGGAGTC,24652,4154,P1,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
P1_Tumor_AAACGGGAGATGTAAC,27554,4334,P1,Tumor,Epithelial,Tumor_KC_Basal,Tumor_KC_Basal
P1_Tumor_AAACGGGAGCTGCAAG,24980,4822,P1,Tumor,Epithelial,Tumor_KC_Cyc,Tumor_KC_Cyc
...,...,...,...,...,...,...,...
P10_Normal_TTTGTCAAGAGTCTGG,5585,1659,P10,Normal,Epithelial,Normal_KC_Diff,Normal_KC_Diff
P10_Normal_TTTGTCAAGTCGTTTG,14207,3541,P10,Normal,LC,LC,LC
P10_Normal_TTTGTCACAGTGGGAT,10068,2540,P10,Normal,LC,LC,LC
P10_Normal_TTTGTCATCCAGGGCT,5223,1849,P10,Normal,LC,LC,LC


In [66]:
# count number of unique spots from metadata
meta_barcds = [x.split('_')[2] for x in metadata.index] # split rownames on '_' and keep barcode only
np.unique(barcds).shape 

(46699,)

In [67]:
meta_barcds

['AAACCTGAGTCAAGCG',
 'AAACCTGCAAATTGCC',
 'AAACCTGGTAGGAGTC',
 'AAACGGGAGATGTAAC',
 'AAACGGGAGCTGCAAG',
 'AAAGATGAGAAGGTTT',
 'AAAGATGCAATAACGA',
 'AAAGATGCAGACTCGC',
 'AAAGATGCAGTCAGCC',
 'AAAGATGGTGGTGTAG',
 'AAAGCAATCCAGTAGT',
 'AAAGTAGCAGACGTAG',
 'AAAGTAGGTGGCTCCA',
 'AAAGTAGTCTTTACGT',
 'AAATGCCAGTTAACGA',
 'AAATGCCCACGAGAGT',
 'AAATGCCTCTACTTAC',
 'AACACGTTCAACACAC',
 'AACACGTTCTCCAACC',
 'AACCATGGTGAGGCTA',
 'AACCATGTCCTTTCTC',
 'AACCGCGCACTTCGAA',
 'AACGTTGTCGTCTGCT',
 'AACTCAGAGTGTACGG',
 'AACTCAGGTCAAACTC',
 'AACTCCCTCGATAGAA',
 'AACTCTTCATACCATG',
 'AACTCTTGTCGTCTTC',
 'AACTCTTGTGGTTTCA',
 'AACTGGTCATTCTTAC',
 'AACTGGTGTTACTGAC',
 'AACTTTCCAGTGGGAT',
 'AACTTTCTCTAACTTC',
 'AAGACCTAGCGATTCT',
 'AAGACCTGTCGCGTGT',
 'AAGACCTGTGATGTGG',
 'AAGGAGCGTCCGTCAG',
 'AAGGCAGAGACTCGGA',
 'AAGGCAGAGGCTCAGA',
 'AAGGTTCAGTCTTGCA',
 'AAGTCTGCACTGAAGG',
 'AATCCAGTCTAGAGTC',
 'AATCGGTCAGAGCCAA',
 'AATCGGTCATTATCTC',
 'ACACCAACATACTCTT',
 'ACACCCTAGTAACCCT',
 'ACACCGGCACGAAGCA',
 'ACAGCCGCATC

In [81]:
# check if meta_barcds in st_barcodes
[i for e in meta_barcds for i in st_barcodes.iloc[:,0] if e in i] 

[]

In [89]:
# check if visium spots are in st_barcodes
[i for e in vs_spots for i in st_barcodes.iloc[:,0] if e in i] 

[]

In [83]:
# sanity check
[i for e in fake.iloc[:,0] for i in st_barcodes.iloc[:, 0] if e in i] 

['GTCCGATATGATTGCCGC', 'GCGACGCGGCATAGTTTA']

In [92]:
# just for fun, see which don't match (get length)
print(len(set(st_barcodes.iloc[:, 0])-{i for e in fake.iloc[:, 0] for i in st_barcodes.iloc[:, 0] if e in i}))
print(len(st_barcodes.iloc[:, 0]))

1932
1934


In [61]:
# cSCC_counts = pd.read_csv(path+'scRNAseq/'+'GSE144236_cSCC_counts.txt', sep = '\t', chunksize=1000)

# # merge all chunks into one df
# scRNAseq = pd.concat([chunk for chunk in cSCC_counts])

# # check if any of the scRNA barcodes are in the st_barcodes
# [i for e in scRNAseq_barcds for i in st_barcodes.iloc[:,0] if e in i] 

In [127]:
scRNAseq

,P1_Tumor_AAACCTGAGTCAAGCG,P1_Tumor_AAACCTGCAAATTGCC,P1_Tumor_AAACCTGGTAGGAGTC,P1_Tumor_AAACGGGAGATGTAAC,P1_Tumor_AAACGGGAGCTGCAAG,P1_Tumor_AAAGATGAGAAGGTTT,P1_Tumor_AAAGATGCAATAACGA,P1_Tumor_AAAGATGCAGACTCGC,P1_Tumor_AAAGATGCAGTCAGCC,P1_Tumor_AAAGATGGTGGTGTAG,...,P10_Normal_TTTGGTTCACTCTGTC,P10_Normal_TTTGGTTCATCATCCC,P10_Normal_TTTGGTTCATCCCATC,P10_Normal_TTTGGTTTCTCTAAGG,P10_Normal_TTTGTCAAGACAATAC,P10_Normal_TTTGTCAAGAGTCTGG,P10_Normal_TTTGTCAAGTCGTTTG,P10_Normal_TTTGTCACAGTGGGAT,P10_Normal_TTTGTCATCCAGGGCT,P10_Normal_TTTGTCATCGAGGTAG
Patient,1,1,1,1,1,1,1,1,1,1,...,10,10,10,10,10,10,10,10,10,10
"Tissue: 0=Normal, 1=Tumor",1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
RP11-34P13.7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AL627309.1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
AP006222.2,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC145205.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BAGE5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CU459201.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC002321.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
scRNAseq_barcds = [x.split('_')[2] for x in scRNAseq.columns.values]
scRNAseq_barcds

['AAACCTGAGTCAAGCG',
 'AAACCTGCAAATTGCC',
 'AAACCTGGTAGGAGTC',
 'AAACGGGAGATGTAAC',
 'AAACGGGAGCTGCAAG',
 'AAAGATGAGAAGGTTT',
 'AAAGATGCAATAACGA',
 'AAAGATGCAGACTCGC',
 'AAAGATGCAGTCAGCC',
 'AAAGATGGTGGTGTAG',
 'AAAGCAATCCAGTAGT',
 'AAAGTAGCAGACGTAG',
 'AAAGTAGGTGGCTCCA',
 'AAAGTAGTCTTTACGT',
 'AAATGCCAGTTAACGA',
 'AAATGCCCACGAGAGT',
 'AAATGCCTCTACTTAC',
 'AACACGTTCAACACAC',
 'AACACGTTCTCCAACC',
 'AACCATGGTGAGGCTA',
 'AACCATGTCCTTTCTC',
 'AACCGCGCACTTCGAA',
 'AACGTTGTCGTCTGCT',
 'AACTCAGAGTGTACGG',
 'AACTCAGGTCAAACTC',
 'AACTCCCTCGATAGAA',
 'AACTCTTCATACCATG',
 'AACTCTTGTCGTCTTC',
 'AACTCTTGTGGTTTCA',
 'AACTGGTCATTCTTAC',
 'AACTGGTGTTACTGAC',
 'AACTTTCCAGTGGGAT',
 'AACTTTCTCTAACTTC',
 'AAGACCTAGCGATTCT',
 'AAGACCTGTCGCGTGT',
 'AAGACCTGTGATGTGG',
 'AAGGAGCGTCCGTCAG',
 'AAGGCAGAGACTCGGA',
 'AAGGCAGAGGCTCAGA',
 'AAGGTTCAGTCTTGCA',
 'AAGTCTGCACTGAAGG',
 'AATCCAGTCTAGAGTC',
 'AATCGGTCAGAGCCAA',
 'AATCGGTCATTATCTC',
 'ACACCAACATACTCTT',
 'ACACCCTAGTAACCCT',
 'ACACCGGCACGAAGCA',
 'ACAGCCGCATC

In [103]:
[i for e in scRNAseq_barcds for i in st_barcodes.iloc[:,0] if e in i] 

[]

In [108]:
# scRNA in metadata? yes.
# [i for e in scRNAseq.columns.values.tolist() for i in metadata.index if e in i] 
print(len([i for e in scRNAseq.columns.values.tolist() for i in metadata.index if e in i]))
print(len(metadata))
print(len(scRNAseq.columns))

48171
48164
48164


In [36]:
metadata['patient'].unique()

array(['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10'],
      dtype=object)

In [37]:
len(scRNA) # number of patients = 10 (as expected)

10

In [117]:
all_counts.head()

,P2_7x45_1,P2_8x30_1,P2_8x32_1,P2_8x34_1,P2_8x36_1,P2_8x38_1,P2_8x44_1,P2_8x46_1,P2_9x29_1,P2_9x31_1,...,P5_41x27_3,P5_41x31_3,P5_41x33_3,P5_41x35_3,P5_41x37_3,P5_41x39_3,P5_41x41_3,P5_42x36_3,P5_42x38_3,P5_42x40_3
Patient,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
Section replicate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
pixel_x,3282.6,3473.4,3471.9,3474.1,3469.5,3467.4,3471.4,3462.3,3650.9,3649.9,...,9448.4,9446.6,9437.3,9451.4,9451.1,9445.3,9451.1,9634.2,9628.1,9623.4
pixel_y,9536.7,6539.5,6937.9,7338.8,7736.4,8137.2,9337.8,9738.9,6336.8,6737.0,...,6984.0,7809.0,8205.3,8613.4,9011.1,9412.5,9810.7,8815.8,9224.7,9620.7
FO538757.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
# similar genes in scRNAseq and ST_all_counts
[i for e in scRNAseq.index[2:] for i in all_counts.index[4:] if e in i]

['SAMD11',
 'NOC2L',
 'KLHL17',
 'PLEKHN1',
 'HES4',
 'ISG15',
 'AGRN',
 'RNF223',
 'C1orf159',
 'TNFRSF18',
 'TNFRSF4',
 'SDF4',
 'B3GALT6',
 'FAM132A',
 'UBE2J2',
 'SCNN1D',
 'ACAP3',
 'PUSL1',
 'CPSF3L',
 'DVL1',
 'MXRA8',
 'AURKAIP1',
 'CCNL2',
 'MRPL20',
 'ANKRD65',
 'VWA1',
 'ATAD3C',
 'ATAD3B',
 'ATAD3A',
 'SSU72',
 'MIB2',
 'MMP23B',
 'CDK11B',
 'SLC35E2B',
 'CDK11A',
 'SLC35E2B',
 'SLC35E2',
 'NADK',
 'NADK2',
 'GNB1',
 'GNB1L',
 'TMEM52',
 'GABRD',
 'PRKCZ',
 'SKI',
 'SKIDA1',
 'GSKIP',
 'CASKIN1',
 'CASKIN2',
 'SKIL',
 'SKIV2L2',
 'SKIV2L',
 'MORN1',
 'RER1',
 'PEX10',
 'PLCH2',
 'PANK4',
 'HES5',
 'TNFRSF14',
 'FAM213B',
 'ARHGEF16',
 'MEGF6',
 'TPRG1L',
 'WRAP73',
 'TP73',
 'SMIM1',
 'SMIM12',
 'SMIM11B',
 'SMIM11A',
 'SMIM14',
 'SMIM15',
 'SMIM13',
 'SMIM19',
 'SMIM10',
 'SMIM10L2B',
 'SMIM10L2A',
 'LRRC47',
 'CEP104',
 'DFFB',
 'C1orf174',
 'AJAP1',
 'NPHP4',
 'KCNAB2',
 'RPL22',
 'RPL22L1',
 'MRPL22',
 'RNF207',
 'ICMT',
 'GPR153',
 'ACOT7',
 'HES2',
 'ESPN',
 'ESPNL',


## Explore sc MIBI files

In [137]:
import cv2
import numpy as np

mibi = [cv2.imread(j) for j in glob.glob(os.path.join(path+'MIBI/'+'*.tiff'))]

In [141]:
np.asarray(mibi[0], dtype=np.float64)

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [142]:
from PIL import Image

mibi = [Image.open(j) for j in glob.glob(os.path.join(path+'MIBI/'+'*.tiff'))]

In [146]:

for i in range(len(mibi)):
    plt.show(mibi[i])

NameError: name 'plt' is not defined